<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Figure 2

_Authors: Oriana Chegwidden and Jeremy Freeman_

The methods below conduct the analyses to recreate Figure 2 included the
manuscript <<< insert doi >>>.


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy as sp
from carbonplan_forests import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
from carbonplan_forests.utils import get_store


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from carbonplan_forests import load, setup, plot, fit, utils, prepare, collect
from showit import image
import scipy as sp
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import regionmask as rm
from scipy.stats import binom
import altair as alt

from palettable.colorbrewer.sequential import YlOrRd_9

In [ ]:
coarsen = 8
store = "az"
tlim = ("1984", "2018")

In [ ]:
historical_ds = xr.open_zarr(get_store("carbonplan-scratch", "data/fire.zarr"))

In [ ]:
mask = load.mask(store=store, year=2001)

In [ ]:
nlcd = load.nlcd(store=store, year=2001, coarsen=coarsen, mask=mask)
nftd = load.nftd(store=store, area_threshold=1500, coarsen=coarsen, mask=mask)

In [ ]:
mtbs = load.mtbs(store=store, coarsen=coarsen, tlim=tlim, mask=mask)
# give it the same x/y coords as nftd (and nlcd and climate)
# this re-assignment will help us with some masking lower down
mtbs = mtbs.assign_coords({"x": nftd.x, "y": nftd.y})

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
plot.fire.evaluation(
    mtbs,
    historical_ds,
    data_var="monthly",
    model_var="historical",
    clim=(0.0001, 0.005),
    add_map=False,
)

In [ ]:
historical_ds = xr.open_zarr(
    get_store("carbonplan-scratch", "data/fire_plus2.zarr")
)
plot.fire.evaluation(
    mtbs,
    historical_ds,
    data_var="monthly",
    model_var="historical",
    clim=(0.0001, 0.005),
    add_map=False,
)

In [ ]:
historical_ds = xr.open_zarr(
    get_store("carbonplan-scratch", "data/fire_plus4.zarr")
)
plot.fire.evaluation(
    mtbs,
    historical_ds,
    data_var="monthly",
    model_var="historical",
    clim=(0.0001, 0.005),
    add_map=False,
)

In [ ]:
future_ts = xr.open_zarr(
    get_store("carbonplan-scratch", "data/fire_future_temp_precip.zarr")
)

In [ ]:
future_ts = future_ts.assign_coords({"lat": mtbs.lat, "lon": mtbs.lon})

In [ ]:
plot.fire.summary(future_ts["CanESM5_ssp245"], clim=(0.0001, 0.02))

In [ ]:
plot.fire.summary(future_ts["MIROC-ES2L_ssp245"], clim=(0.0001, 0.02))

In [ ]:
plot.fire.summary(future_ts["CanESM5_ssp370"], clim=(0.0001, 0.02))

In [ ]:
plot.fire.summary(future_ts["CanESM5_ssp245"], clim=(0.0001, 0.02))

In [ ]:
from scipy.stats import zscore

In [ ]:
avg = climate.mean(dim=["x", "y"])
plt.plot(zscore(avg.tmean.groupby("time.year").mean()))
plt.plot(
    zscore(
        future_ts["CanESM5_ssp370"].groupby("time.year").sum().mean(["x", "y"])
    )
)

In [ ]:
avg = climate.mean(dim=["x", "y"])
plt.plot(zscore(avg.tmean.groupby("time.year").mean()))
plt.plot(
    zscore(
        future_ts["CanESM5_ssp245"].groupby("time.year").sum().mean(["x", "y"])
    )
)

In [ ]:
toplot = (
    future_ts.groupby("time.year")
    .sum()
    .coarsen(year=5)
    .mean()
    .mean(dim=["x", "y"])
    .compute()
)  # ['MIROC-ES2L_ssp245']

In [ ]:
line_style = {"ssp245": "dotted", "ssp370": "dashed", "ssp585": "solid"}
colors = {
    "CanESM5": "black",
    "MRI-ESM2-0": "red",
    "MIROC-ES2L": "orange",
    "MIROC6": "blue",
    "FGOALS-g3": "green",
    "HadGEM3-GC31-LL": "grey",
}
plt.figure(figsize=(12, 8))
for scenario in ["ssp245", "ssp370", "ssp585"]:
    for gcm in [
        "CanESM5",
        "MRI-ESM2-0",
        "MIROC-ES2L",
        "MIROC6",
        "FGOALS-g3",
        "HadGEM3-GC31-LL",
    ]:
        run_name = "{}_{}".format(gcm, scenario)
        try:
            toplot[run_name].plot(
                label=run_name,
                linestyle=line_style[scenario],
                color=colors[gcm],
            )
        except:
            print("{} nope".format(run_name))
plt.ylabel("fire risk")
plt.ylim(0, 0.01)
plt.legend()

In [ ]:
figure = alt.vconcat()
for scenario in ["ssp245", "ssp370", "ssp585"]:
    for gcm in [
        "CanESM5",
        "MRI-ESM2-0",
        "MIROC-ES2L",
        "MIROC6",
        "FGOALS-g3",
        "HadGEM3-GC31-LL",
    ]:
        run_name = "{}_{}".format(gcm, scenario)
        try:
            panel = plot.fire.summary(
                future_ts[run_name], clim=(0.0001, 0.08), title=run_name
            )
            figure &= panel
        except:
            print("nope!")

In [ ]:
figure

# Let's check all the z-scores


In [ ]:
for scenario in ["ssp245", "ssp370", "ssp585"]:
    for (gcm, member) in [
        ("CanESM5", "r10i1p1f1"),
        ("UKESM1-0-LL", "r10i1p1f2"),
        ("MRI-ESM2-0", "r1i1p1f1"),
        ("MIROC-ES2L", "r1i1p1f2"),
        ("MIROC6", "r10i1p1f1"),
        ("FGOALS-g3", "r1i1p1f1"),
        ("HadGEM3-GC31-LL", "r1i1p1f3"),
    ]:
        try:
            run_name = "{}_{}".format(gcm, scenario)
            #         try:
            climate = load.cmip(
                store="az",
                model=gcm,
                coarsen=8,
                scenario=scenario,
                tlim=("2015", "2099"),
                variables=["tmean", "ppt"],
                sampling="monthly",
                member=member,
            ).mean(dim=["x", "y"])
            plt.figure(figsize=(6, 5))
            plt.plot(
                zscore(climate.tmean.groupby("time.year").mean()), label="temp"
            )
            plt.plot(
                zscore(climate.ppt.groupby("time.year").mean()), label="ppt"
            )
            plt.plot(
                zscore(
                    future_ts[run_name]
                    .groupby("time.year")
                    .sum()
                    .mean(["x", "y"])
                ),
                label="fire",
            )
            plt.title(run_name)
            plt.legend()
        except:
            print("nope")